In [40]:
import pandas as pd
import json
from pprint import pprint
import ast

In [32]:
# sep: str | None = ...,
DELIMITER = "|"
# header: int | Sequence[int] | Literal['infer'] | None = ...,
# names: ListLikeHashable[Unknown] | None = ...,
INDEX_COL = False
QUOTE = '"'
# usecols: UsecolsArgType[HashableT@read_csv] = ...,
# dtype: DtypeArg | defaultdict[Unknown, Unknown] | None = ...,
# engine: CSVEngine | None = ...,
# converters: Mapping[int | str, (str) -> Any] | Mapping[int, (str) -> Any] | Mapping[str, (str) 

In [46]:
csv_file = '/mnt/Code/code/AI/agentic-AI/SQL-QA/logs/turn_log.csv'

In [47]:
df = pd.read_csv(csv_file, delimiter=DELIMITER, index_col=INDEX_COL, quotechar=QUOTE)

In [48]:
df

,created_date,question,linking_structured_result,filtered_schema_tables,direct_generation_prompt,cot_generation_prompt,direct_generation_generation_response,query_validation_prompt,cot_generation_generation_response,query_validation_response,direct_generation,cot_generation,merger_prompt,merger_response,merger_result,strategy,final_state
0,2025-06-20 11:05:02,khách hàng nào chi tiêu nhiều nhất trong tháng 4,{'schema_linking': 'patient:Bảng này chứa thôn...,"[\n ""sale_invoice"",\n ""expense_voucher"",...",---retry--- \n \n\n **CHÚ Ý**: \n - Phải...,---retry--- \n \n\n **CHÚ Ý**: \n - Phải...,---retry--- \n {'explaination': 'This SQL quer...,---retry--- \n \n\n **CHÚ Ý**: \n - Phải...,---retry--- \n {'explaination': 'The query ret...,"---retry--- \n {'is_sql_correct': True, 'expla...",Generator direct_generation result: False.\nDe...,Generator cot_generation result: False.\nDetai...,\n\n **CHÚ Ý**: \n - Phải tuân thủ đúng ...,{'explaination': 'This SQL query selects the ...,{'explaination': 'This SQL query selects the f...,"[{'strategy': 'direct_generation', 'thoughts':...",NaN
1,2025-06-20 13:38:45,khách hàng nào chi tiêu nhiều nhất trong tháng 4,"{'tables': ['patient', 'sale_invoice', 'servic...","[\n ""sale_invoice"",\n ""receipt"",\n ""r...",---retry--- \n \n\n **CHÚ Ý**: \n - Phải...,---retry--- \n \n\n **CHÚ Ý**: \n - Phải...,---retry--- \n {'sql': 'SELECT p.id AS patient...,---retry--- \n \n\n **CHÚ Ý**: \n - Phải...,"---retry--- \n {'sql': 'SELECT p.full_name, SU...","---retry--- \n {'is_sql_correct': True, 'expla...",Generator direct_generation result: False.\nDe...,Generator cot_generation result: False.\nDetai...,\n\n **CHÚ Ý**: \n - Phải tuân thủ đúng ...,"{'sql': 'SELECT p.full_name, SUM(si.total_amo...","{'sql': 'SELECT p.full_name, SUM(si.total_amou...","[{'strategy': 'direct_generation', 'thoughts':...","{'schema_linking': {'GSV': Schema(name='GSV', ..."


In [35]:
print(
        df.at[0, 'merger_prompt']
        )



    **CHÚ Ý**: 
    - Phải tuân thủ đúng cú pháp và các quy tắc của hệ quản trị cơ sở dữ liệu MYSQL.
    - Câu truy vấn SQL phải chứa thông tin có ý nghĩa và dễ hiểu cho người dùng.
    
    

**Mô tả nhiệm vụ**:
Bạn là một chuyên gia SQL có nhiệm vụ tổng hợp các câu lệnh truy vấn SQL candidate thành một câu lệnh SQL cuối cùng. Dựa trên câu hỏi của người dùng, lược đồ cơ sở dữ liệu, và các câu lệnh SQL candidate được cung cấp, hãy phân tích và chọn ra câu lệnh SQL tốt nhất hoặc kết hợp các phần tốt nhất từ các candidate để tạo ra câu lệnh SQL cuối cùng.

**Quy trình**:
1. **Phân tích câu hỏi**: Đọc kỹ câu hỏi của người dùng để hiểu rõ yêu cầu.
2. **Xem xét lược đồ cơ sở dữ liệu**: Kiểm tra cấu trúc cơ sở dữ liệu để đảm bảo câu lệnh SQL cuối cùng phù hợp với lược đồ.
3. **Đánh giá các câu lệnh SQL candidate**:
   - Kiểm tra tính chính xác của cú pháp.
   - Đánh giá hiệu suất và tối ưu hóa.
   - Xem xét khả năng cung cấp thông tin dễ hiểu và có ý nghĩa cho người dùng.
4. **Tổng hợp câu

In [50]:

merge_resp = ast.literal_eval(df.at[len(df)-1, 'merger_response'].strip())

In [51]:
# (json.loads(df.at[0, 'merger_response'].strip()))
# pprint(df.at[0, 'merger_response'].strip())
pprint(merge_resp)

{'explaination': 'The SQL query selects the full name of the patient and the '
                 'total amount spent by each patient in April, then orders the '
                 'result by the total amount spent in descending order and '
                 'limits the result to the top 1 patient.',
 'sql': 'SELECT p.full_name, SUM(si.total_amount) AS total_spent FROM patient '
        'p JOIN service_invoice si ON p.id = si.patient_id WHERE '
        'MONTH(si.paid_date) = 4 GROUP BY p.full_name ORDER BY total_spent '
        'DESC LIMIT 1;'}


In [55]:
print((df.at[len(df)-1, 'final_state'].strip()))

{'schema_linking': {'GSV': Schema(name='GSV', description=None, tables=[Table(name='sale_invoice', description=None, columns=[Column(name='created_on', type='TIMESTAMP', description='Thời điểm tạo bản ghi,', example='[2025-03-12 13:21:04.825326]'), Column(name='modified_on', type='TIMESTAMP', description='Thời điểm sửa đổi bản ghi gần nhất,', example='[2025-03-24 00:21:33.660689]'), Column(name='created_by', type='INTEGER', description='ID của người tạo bản ghi,', example='[1]'), Column(name='modified_by', type='INTEGER', description='ID của người sửa đổi bản ghi gần nhất,', example='[1]'), Column(name='deleted_at', type='TIMESTAMP', description='Thời điểm bản ghi bị xóa (nếu có)', example=None), Column(name='creator', type='VARCHAR', description='Tên người tạo bản ghi,', example='[Admin]'), Column(name='modifier', type='VARCHAR', description='Tên người sửa đổi bản ghi gần nhất,', example='[Admin]'), Column(name='id', type='INTEGER', description='Mã định danh duy nhất của hóa đơn bán h

In [52]:
df.columns

Index(['created_date', 'question', 'linking_structured_result',
       'filtered_schema_tables', 'direct_generation_prompt',
       'cot_generation_prompt', 'direct_generation_generation_response',
       'query_validation_prompt', 'cot_generation_generation_response',
       'query_validation_response', 'direct_generation', 'cot_generation',
       'merger_prompt', 'merger_response', 'merger_result', 'strategy',
       'final_state'],
      dtype='object')